In [2]:
from src.code_gen.code_generator import CCodeGenerator
from src.lexer.hulk_lexer import HulkLexer
from src.parser.hulk_parser import HulkParser
from src.hulk_grammar.hulk_grammar import G
from src.evaluation import evaluate_reverse_parse
from src.semantics.semantic_analysis_pipeline import semantic_analysis_pipeline

lexer = HulkLexer(True)
parser = HulkParser()

In [7]:
tokens, errors = lexer('''  
    function Sort(A) => 
            let  aux = 0 in for (i in range(0, A.size()))
                for (j in range(i, A.size()))
                 if(A[j] < A[i])
                 {
                    aux := A[i];
                    A[i] := A[j];
                    A[j] := aux;
                    A;
                }
                else A;
                       
    print(Sort([78, 12, 100, 0, 6, 9, 4.5]));
    ''')

derivation, operations, errors = parser(tokens)
print(errors)
ast = evaluate_reverse_parse(derivation, operations, tokens)
ast, semantic_errors, context, scope = semantic_analysis_pipeline(ast, False)

generator = CCodeGenerator()
code = generator(ast, context)

with open("test.c", "w") as out:
    out.write(code)

print(semantic_errors)

[]
[SemanticError: Cannot infer type of parameter 'A' in 'Sort'. Please specify it., SemanticError: Cannot infer return type of 'Sort'. Please specify it.]


In [8]:
import subprocess

subprocess.run(["gcc", "test.c", "-o", "test.exe"], shell=True)
subprocess.run(['start', 'cmd', '/k', 'test.exe'], shell=True)

CompletedProcess(args=['start', 'cmd', '/k', 'test.exe'], returncode=0)